# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [2]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [3]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [4]:
q = '''
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31';
'''
letter = pd.read_sql(q, conn)
letter

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [5]:
# Your code here
query = '''
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN(SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31');
'''
vary = pd.read_sql(query, conn)
vary

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [6]:
# Your code here
q = '''
SELECT p.productName, p.productDescription, SUM(od.quantityOrdered) AS total_items_sold
FROM products AS p
JOIN orderdetails AS od ON p.productCode = od.productCode
GROUP BY p.productName, p.productDescription
ORDER BY total_items_sold DESC;
'''
sort = pd.read_sql(q, conn)
sort

,productName,productDescription,total_items_sold
0,1992 Ferrari 360 Spider red,"his replica features opening doors, superb det...",1808
1,1937 Lincoln Berline,"Features opening engine cover, doors, trunk, a...",1111
2,American Airlines: MD-11S,Polished finish. Exact replia with official lo...,1085
3,1941 Chevrolet Special Deluxe Cabriolet,"Features opening hood, opening doors, opening ...",1076
4,1930 Buick Marquette Phaeton,"Features opening trunk, working steering system",1074
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,Features include opening and closing doors. Co...,855
105,1911 Ford Town Car,"Features opening hood, opening doors, opening ...",832
106,1936 Mercedes Benz 500k Roadster,This model features grille-mounted chrome horn...,824
107,1970 Chevy Chevelle SS 454,"This model features rotating wheels, working s...",803


In [11]:
q = '''
SELECT p.productName, p.productDescription, 
       (SELECT SUM(od.quantityOrdered) 
        FROM orderdetails AS od 
        WHERE od.productCode = p.productCode) AS total_items_sold
FROM products AS p
ORDER BY total_items_sold DESC;
'''
sold = pd.read_sql(q, conn)
sold

,productName,productDescription,total_items_sold
0,1992 Ferrari 360 Spider red,"his replica features opening doors, superb det...",1808.0
1,1937 Lincoln Berline,"Features opening engine cover, doors, trunk, a...",1111.0
2,American Airlines: MD-11S,Polished finish. Exact replia with official lo...,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,"Features opening hood, opening doors, opening ...",1076.0
4,1930 Buick Marquette Phaeton,"Features opening trunk, working steering system",1074.0
...,...,...,...
105,1911 Ford Town Car,"Features opening hood, opening doors, opening ...",832.0
106,1936 Mercedes Benz 500k Roadster,This model features grille-mounted chrome horn...,824.0
107,1970 Chevy Chevelle SS 454,"This model features rotating wheels, working s...",803.0
108,1957 Ford Thunderbird,"This 1:18 scale precision die-cast replica, wi...",767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [12]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q = '''
SELECT p.productName, 
       (SELECT COUNT(DISTINCT o.customerNumber) 
        FROM orders AS o 
        JOIN orderdetails AS od ON o.orderNumber = od.orderNumber
        WHERE od.productCode = p.productCode) AS total_customers
FROM products AS p
ORDER BY total_customers DESC;
'''
dist = pd.read_sql(q, conn)
dist

,productName,total_customers
0,1992 Ferrari 360 Spider red,40
1,1952 Alpine Renault 1300,27
2,1972 Alfa Romeo GTA,27
3,1934 Ford V8 Coupe,27
4,Boeing X-32A JSF,27
...,...,...
105,1949 Jaguar XK 120,18
106,1952 Citroen-15CV,18
107,1969 Chevrolet Camaro Z28,18
108,2002 Chevy Corvette,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [13]:
# Your code here
q = '''
SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
FROM employees AS e
JOIN offices AS o ON e.officeCode = o.officeCode
WHERE e.employeeNumber IN (
    SELECT DISTINCT c.salesRepEmployeeNumber
    FROM customers AS c
    JOIN orders AS odr ON c.customerNumber = odr.customerNumber
    JOIN orderdetails AS od ON odr.orderNumber = od.orderNumber
    WHERE od.productCode IN (
        SELECT productCode
        FROM orderdetails AS od
        JOIN orders AS o ON od.orderNumber = o.orderNumber
        GROUP BY od.productCode
        HAVING COUNT(DISTINCT o.customerNumber) < 20
    )
);
'''
lent = pd.read_sql(q, conn)
lent

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [14]:
# Your code here
q = '''
SELECT e.employeeNumber, e.firstName, e.lastName, 
       (SELECT COUNT(c.customerNumber)
        FROM customers AS c
        WHERE c.salesRepEmployeeNumber = e.employeeNumber) AS num_customers
FROM employees AS e
WHERE e.employeeNumber IN (
    SELECT c.salesRepEmployeeNumber
    FROM customers AS c
    GROUP BY c.salesRepEmployeeNumber
    HAVING AVG(c.creditLimit) > 15000
)
ORDER BY num_customers DESC;
'''
Dis = pd.read_sql(q, conn)
Dis

,employeeNumber,firstName,lastName,num_customers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7
5,1370,Gerard,Hernandez,7
6,1165,Leslie,Jennings,6
7,1166,Leslie,Thompson,6
8,1188,Julie,Firrelli,6
9,1216,Steve,Patterson,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!